<a href="https://colab.research.google.com/github/BroDeb/BroDeb/blob/main/tsfresh_featureselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import sys
#sys.path.insert(1,r'G:\Shared drives\Project - ML for Ventricular MAPs\Python Scripts')


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

date = datetime.datetime.now()
day = str(date.day)
month = str(date.month)
year = str(date.year)
if len(day) <= 1:
    day = '0' + day
if len(month) <= 1:
    month = '0' + month
    
date = year+month+day


'''3sd over/undershoot correction'''
file = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/Python Scripts/single_lead1_dict_mean_3SD_cutoff_zNorm_pos0_20191111.npz'
'''VTVF Outcome'''
excelSheetPath = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/2019 1111 Final_codes_and_data/Step_2_KCrossValidationSplits/VTVF 3 year/2019 1108 VTVF Split.xlsx'
output_label = 'VT_VF in 3Y'

'''Death Outcome'''
excelSheetPath_2 = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/2019 1111 Final_codes_and_data/Step_2_KCrossValidationSplits/Death 3 year/2019 1108 death 3 year split.xlsx'
output_label_2 = 'Death in 3Y'



#Load preprocessed single MAP data from npz file
data = np.load(file, allow_pickle = True)
if file.endswith('.npz'):
    data = data['arr_0']
data = data.item()


def getDataFrame(key,value,output):
    x = value
    x_reshaped = np.reshape(x,(x.shape[0]*x.shape[1],1))
    ids = np.reshape(np.asarray([[float(key)*10000 + i]*x.shape[1] for i in range(x.shape[0])]),(x.shape[0]*x.shape[1],1))
    time = np.reshape([i for i in range(x.shape[1])]*x.shape[0],(np.prod(x.shape),1))
    ids_time = np.append(ids,time,axis = 1)
    x = np.append(ids_time,x_reshaped,axis = 1)
    x_df = pd.DataFrame(data = x, columns = ['id','time','data points'])#,[str(i) for i in range(x.shape[0]-2)])))
    ids = np.asarray([[float(key)*10000 + i] for i in range(value.shape[0])])
    y = pd.DataFrame(np.append(ids,np.ones((value.shape[0],1))*output,axis = 1),columns = ['id','outcome'])
        
    return  x_df, y

#Load excel sheet into pandas format
long_term_outcome_all = pd.ExcelFile(excelSheetPath) 

#read each sheet into a pd data frame
all_df = pd.DataFrame()
all_y = pd.DataFrame()
for sheet in ['CV1']:#,'CV2','CV3','CV4','CV5','CV6','CV7','CV8','CV9','CV10']:
    print(sheet)
    sheet_name = sheet
    long_term_outcome = pd.read_excel(long_term_outcome_all, sheet_name = sheet)

    
    for index, row in long_term_outcome.iterrows():
        if True: #index > 0 and index =< 47:
            split_num = row['split_num']
            output = row[output_label]
            patient_name = str(row['patient'])
            if patient_name.endswith('_B'):
                patient_name = patient_name[:-2] + '.2'
            if patient_name != '10027':
              x_df,y = getDataFrame(patient_name,data[str(row['patient'])],output)
              all_df = all_df.append(x_df)
              all_y = all_y.append(y)
    
all_df.to_csv(date+"VTVFSingleECGBeatsTsfreshFormat.csv", index = False)
all_y.to_csv(date+"VTVF3YR_outcome.csv", index = False)
            
all_df.to_csv(date+"DeathSingleECGBeatsTsfreshFormat.csv", index = False)
all_y.to_csv(date+"Death3YR_outcome.csv", index = False)

CV1


In [ ]:
import datetime 
import os
#!pip install tsfresh

date = datetime.datetime.now()
day = str(date.day)
month = str(date.month)
year = str(date.year)
if len(day) <= 1:
    day = '0' + day
if len(month) <= 1:
    month = '0' + month
    
date = year+month+day


import pandas as pd
root_dir = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/2019 1111 Final_codes_and_data'
#import tsfresh formated time series 
#timeseries_csv_filePath = os.path.join(root_dir,r'/content/20211125DeathSingleECGBeatsTsfreshFormat.csv')
timeseries_df = pd.read_csv('/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/20211125DeathSingleECGBeatsTsfreshFormat.csv')
#timeseries_df = timeseries_df.set_index('id')

#import tsfresh formatted y labels 
#death3yr_y_labels_path = os.path.join(root_dir,r'Step_3_ChooseTsFeatures/Step_3_1_SaveDataIntoTsfreshFormat/20191116Death3YR_outcome.csv')
death3yr_y_labels = pd.read_csv('/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/20211125Death3YR_outcome.csv')
#vtvf3yr_y_labels_path = os.path.join(root_dir,r'Step_3_ChooseTsFeatures/Step_3_1_SaveDataIntoTsfreshFormat/20191116VTVF3YR_outcome.csv')
vtvf3yr_y_labels = pd.read_csv('/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/20211125VTVF3YR_outcome.csv')

#Initial path to save extracted features or
extracted_features_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/extractedfeatures.csv'

from tsfresh import extract_features
try:
    extracted_features = pd.read_csv(extracted_features_path, index_col = 'id')
except:
    extracted_features = extract_features(timeseries_df, column_id='id', column_sort='time')
    extracted_features.to_csv(extracted_features_path)




Feature Extraction: 100%|██████████| 5/5 [31:44<00:00, 380.81s/it]


In [ ]:
extracted_features_path = '/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/ECG/extractedfeatures.csv'
extracted_features.to_csv(extracted_features_path)

In [ ]:
import sys
sys.path.insert(1,r'/content/drive/Shareddrives/XPublished  - 2021 AJ Mahmood Circ Res ML for Ventricular MAPs/2019 1111 Final_codes_and_data/Step_3_ChooseTsFeatures/Step_3_2_ExtractTSFreshFeaturesLabel')
from drop_corr_features import drop_input_corr_columns


In [ ]:
from tsfresh.utilities.dataframe_functions import impute

#replace NAN, -inf, +inf by median, min, max
impute(extracted_features)

#Sort all extracted features dataframe and Ylabel series based on id so that they are aligned
death3yr_y_series = pd.Series(data = death3yr_y_labels['outcome'].values, index=death3yr_y_labels['id'])
death3yr_y_series = death3yr_y_series.sort_index(axis = 0)

vtvf3yr_y_series = pd.Series(data = vtvf3yr_y_labels['outcome'].values, index=vtvf3yr_y_labels['id'])
vtvf3yr_y_series = vtvf3yr_y_series.sort_index(axis = 0)

extracted_features = extracted_features.sort_index(axis = 0)
#extracted_features = extracted_features.set_index('id')

death3yr_y_series.to_csv(r'TsfreshDeath3yr_y_series'+date+'.csv', header = True)
vtvf3yr_y_series.to_csv(r'TsfreshVtvf3yr_y_series'+date+'.csv', header = True)
extracted_features.to_csv(r'TsfreshExtractedFeaturesBlindedLabelImputed'+date+'.csv', header = True)


from tsfresh import select_features

for i in range(2):
    #use tsfresh labels to choose relevant features 
    if i == 0:
        features_filtered = select_features(extracted_features, vtvf3yr_y_series, ml_task='classification')
    elif i == 1:
        features_filtered = select_features(extracted_features, death3yr_y_series, ml_task='classification')
    
    #drop all but one feature with correlation 0.9 or more
    from drop_corr_features import drop_input_corr_columns
    features_filtered_no_colnr, to_drop = drop_input_corr_columns(features_filtered, corr_fac = 0.9)
    
    #drop any columns with with nan values
    features_filtered_no_colnr = features_filtered_no_colnr.dropna(axis='columns')
    
    #Apply z-norm to all features column-wise
    features_mean = features_filtered_no_colnr.mean()
    features_std = features_filtered_no_colnr.std()
    features_filtered_no_colnr_norm=((features_filtered_no_colnr-features_mean)/features_std)
    
    if i == 0:
        features_filtered_no_colnr_norm.to_csv(r'ECGTsfreshExtractedFeaturesVtvf3YRLabel'+date+'.csv')
    elif i == 1:
        features_filtered_no_colnr_norm.to_csv(r'ECGTsfreshExtractedFeaturesDeath3YRLabel'+date+'.csv')


/usr/local/lib/python3.7/dist-packages/tsfresh/utilities/dataframe_functions.py:172: RuntimeWarning: The columns ['data points__query_similarity_count__query_None__threshold_0.0'] did not have any finite values. Filling with zeros.
  df.iloc[:, np.where(is_col_non_finite)[0]].columns.values), RuntimeWarning)


In [ ]:
print(set(data.keys()))

{'10007', '10030', '10028', '10001', '10032_B', '10062', '10048_B', '10063', '10055', '10026', '10065', '10053', '10051', '10050_B', '10037', '10064', '10049', '10006', '10061', '10020', '10068', '10044', '10039', '10035', '10032', '10023', '10001_B', '10040', '10024', '10054', '10033', '10016', '10015', '10042', '10003', '10034', '10069', '10052', '10075', '10038', '10072', '10045', '10048', '10028_B', '10019', '10050'}
